In [1]:
import os
import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.externals import joblib
from joblib import dump, load

from sklearn.metrics import mean_squared_error, r2_score

In [2]:
def load_model_predict(data_input,safety_factor): 
    
#     today_string = datetime.datetime.today().strftime('%Y_%m_%d')
    model = load('model/close_Δt_modelSelected_'+ today_string + '.joblib') 
    
    scaler_file = "model/close_Δt_model_scaler.save"
    scaler = joblib.load(scaler_file) 
    
    data_input_transform = scaler.transform(data_input)
    Δt_pred = model.predict(data_input_transform) + safety_factor*60
    
    return Δt_pred

In [3]:
def load_test_data(X_path, y_path, safety_factor):
    
#     global X_input, y_input
    X_input = pd.read_csv(X_path).set_index(keys = '关机时间')
    y_input = pd.read_csv(y_path, header=None)[1]
    
    y_pred = load_model_predict(X_input,safety_factor)
    mse_score = mean_squared_error(y_pred/60, y_input/60)
    
    df_y_input = pd.DataFrame(data=y_input.values, index=X_input.index, columns=['Δt_true'])
    df_y_pred = pd.DataFrame(data=y_pred, index=X_input.index, columns=['Δt_predict'])
    
    return df_y_pred, df_y_input, X_input, mse_score

In [4]:
def test_HongSiEntryEndTime_transform(zhisi_path,df_concat):

    hongsi = pd.read_csv(zhisi_path, encoding='utf-8')
    hongsi.EntryEndTime = pd.to_datetime(hongsi.EntryEndTime)
    
    # 根据每一个预测的脱离时间点，找到前后的
    hongsi_last_time_pred =[]
    for i in range(len(df_concat.Δt_predict)):
        hongsi_end=hongsi['EntryEndTime'][hongsi['EntryEndTime'] <= df_concat['预测脱离标准时间点'][i]].max()
        hongsi_last_time_pred.append(hongsi_end)

    hongsi_end_next_pred=[]
    for i in range(len(hongsi_last_time_pred)):
        hongsi_end_next_pred.append(hongsi['EntryEndTime'][hongsi['EntryEndTime'] > hongsi_last_time_pred[i]].min())

    hongsi_end_next2_pred=[]
    for i in range(len(hongsi_end_next_pred)):
        hongsi_end_next2_pred.append(hongsi['EntryEndTime'][hongsi['EntryEndTime'] > hongsi_end_next_pred[i]].min())

    hongsi_lasttime=[]
    for i in range(len(df_concat)):
        if (hongsi_end_next_pred[i] - hongsi_last_time_pred[i]) >= pd.Timedelta(hours=12):
            hongsi_lasttime.append(hongsi_last_time_pred[i])
        elif (hongsi_end_next2_pred[i]-hongsi_end_next_pred[i]) >= pd.Timedelta(hours=12):
            hongsi_lasttime.append(hongsi_end_next_pred[i])

    df_concat['哄切丝_最后开始时间'] = hongsi_lasttime
    
    return df_concat

In [5]:
def test_result_pbi(X_path,y_path,safety_factor,zhisi_path,df_concat_test):

    df_concat_test['现在时间'] = pd.to_timedelta(df_concat_test['Δt_predict'],unit='s').index.astype('datetime64[ns]')

    df_concat_test['Δt_predict'] = pd.to_timedelta(df_concat_test['Δt_predict'],unit='s')
    df_concat_test['Δt_true'] = pd.to_timedelta(df_concat_test['Δt_true'],unit='s')

    # 增加脱离标准时间点维度
    df_concat_test['预测脱离标准时间点'] = df_concat_test['Δt_predict'] + df_concat_test['现在时间']
    df_concat_test['实际脱离标准时间点'] = df_concat_test['Δt_true'] + df_concat_test['现在时间']
    
    # 引入烘丝计划时间
    df_concat_test = test_HongSiEntryEndTime_transform(zhisi_path,df_concat_test)
    df_concat_test.to_csv('csv_pbi/close_data_' + today_string + '.csv',index=True)
    
    return df_concat_test

In [6]:
# (X_input.index.astype('datetime64[ns]')+ pd.to_timedelta(df_y_pred['Δt_predict'],unit='s')).reset_index()[0][0]

In [7]:
def real_time_pred(df_X_realtime,safety_factor,EntryEndTime):
        
    y_pred = load_model_predict(df_X_realtime,safety_factor)
    df_y_pred = pd.DataFrame(data=y_pred, index=df_X_realtime, columns=['Δt_predict'])
    
    timestamp_pred = (X_input.index.astype('datetime64[ns]')+ pd.to_timedelta(df_y_pred['Δt_predict'],unit='s')).reset_index()[0]
    
    if timestamp_pred <= EntryEndTime:
        action = 1  # 1代表继续开
    else:
        action = 0  # 0代表关闭
    
    return action

In [8]:
if __name__=="__main__":
    
    global today_string
    today_string = datetime.datetime.today().strftime('%Y_%m_%d')
    
    X_path = 'data_X_y/Close_X_'+ today_string + '.csv'
    y_path = 'data_X_y/Close_y_'+ today_string + '.csv'
    safety_factor = 0.5
    zhisi_path = "data\product_plan\hongyesi_order0529.csv"
    
    df_y_pred, df_y_input, X_input, mse_score = load_test_data(X_path, y_path, safety_factor)
    df_concat_test=pd.concat([X_input,df_y_input,df_y_pred],axis=1)
    
    df_concat_test = test_result_pbi(X_path,y_path,safety_factor,zhisi_path,df_concat_test)
    
    # df_X_realtime =XXXX
    # EntryEndTime = XXXX
    # action = real_time_pred(df_X_realtime,safety_factor,EntryEndTime)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  if __name__ == '__main__':
